# PNN

Eileen Zhang 2020/8/20

![PNN](https://img-blog.csdnimg.cn/20190516094615674.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzE4MjkzMjEz,size_16,color_FFFFFF,t_70)

**注: 
Paper https://arxiv.org/abs/1611.00144  
如下算法 内积,外积 使用einsum 实现,来提高效率**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.layers import Layer,Dense,Embedding,Dropout,BatchNormalization,Dot

In [3]:
from tensorflow.keras.models import Model

In [4]:
class FeaturesEmbedding(Layer):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = Embedding(np.sum(field_dims), embed_dim)
        self.offsets = tf.constant(np.expand_dims(np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.float32),0))
    
    def build(self, input_shape):    
        super().build(input_shape)
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + self.offsets
        return self.embedding(x)

In [5]:
class MultiLayerPerceptron(Layer):
    def __init__(self, units, dropout = 0.8, output_layer=True):
        super().__init__()
        self.output_layer = output_layer
        self.dropout = Dropout(dropout)
        self.bns = [BatchNormalization() for i in range(len(units))]
        self.denses = [Dense(u,activation = 'relu') for u in units]
        if output_layer:
            self.denses.append(Dense(1))
     
    def build(self, input_shape):    
        super().build(input_shape)
    
    def call(self, inputs, training=False):
        for bn, dense in zip(self.bns,self.denses) :
            inputs = bn(inputs,training)
            inputs = dense(inputs)
            if training:
                inputs = self.dropout_layer(inputs, training=training)
        if self.output_layer:
            inputs = self.denses[-1](inputs)
        return inputs

In [6]:
class ProductNeuralNetworkModel(Model):
    """
    A Keras implementation of inner/outer Product Neural Network.
    Reference:
        Y Qu, et al. Product-based Neural Networks for User Response Prediction, 2016.
    """

    def __init__(self, field_dims, embed_dim, mlp_units = [100,20], dropout = 0.8, method='inner'):
        super().__init__()
        num_fields = len(field_dims)
        self.method = method
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.mlp = MultiLayerPerceptron(mlp_units, dropout)
        self.fc = Dense(1, activation = 'sigmoid')
        self.outer_squeeze = Dense(1)
        
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        embed_x = self.embedding(x)
        
        # einsum 更快
        if self.method == 'inner':
            embed_x = tf.einsum('mij->mji', embed_x)
            cross_term = tf.einsum('mni,mni->mn', embed_x, embed_x)
            cross_term = tf.expand_dims(cross_term,-1)
        elif self.method == 'outer':
            cross_term = tf.einsum('mni,mnj->mnij', embed_x, embed_x)
            embed_x = tf.expand_dims(embed_x, -1)
        else:
            raise ValueError('unknown product type: ' + method)
        x = tf.concat([embed_x,cross_term], -1)
        x = self.mlp(x)
        x = tf.squeeze(x,-1)
        
        if self.method == 'outer':           
            x = self.outer_squeeze(x)
            x = tf.squeeze(x,-1)
        x = self.fc(x)
        return x


In [7]:
model = ProductNeuralNetworkModel([10,20,40,41,51],2)

In [8]:
model.build

<bound method Model.build of <__main__.ProductNeuralNetworkModel object at 0x7f83f22d2fd0>>

In [9]:
test = tf.constant([[0.,1.,2.,3.,4.],[10.,20.,30.,40.,50.]])
test

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [10., 20., 30., 40., 50.]], dtype=float32)>

In [10]:
model(test)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.49990624],
       [0.5019269 ]], dtype=float32)>

In [11]:
model = ProductNeuralNetworkModel([10,20,40,41,51], 2, method='outer')

In [12]:
model(test)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.5000039 ],
       [0.50121194]], dtype=float32)>